# Molten Salt Comparison

In this tutorial we will compare two large molten salt simulation of NaCl and KCl to see how their structures and properties differ from one another.

This tutorial assumes that you are already familiar with the general MDSuite interface and have looked over the previous tutorial.

In [ ]:
import mdsuite as mds
import matplotlib.pyplot as plt
from zinchub import DataHub
import tensorflow as tf
import numpy as np
from scipy.integrate import cumtrapz

In this tutorial we are using two, 1000 atom simulations of NaCl and KCl run for 5000 time steps. Keep in mind, due to the size of these files (~140 MB each) they may take some time to download and unzip (within a minute depending on internet connection).

In [ ]:
NaCl_file = DataHub(url="https://github.com/zincware/DataHub/tree/main/NaCl_rnd_md", tag="v0.1.0")
KCl_file = DataHub(url="https://github.com/zincware/DataHub/tree/main/KCl_rnd_md", tag="v0.1.0")

KCl_data = KCl_file.get_file('.')[0]
NaCl_data = NaCl_file.get_file('.')[0]

In [ ]:
md_project = mds.Project(name="Molten_Salt_Comparison", storage_path='.')

Again, depending on your computer, adding the data may take up between 25s to 1m total.

In [ ]:
NaCl_experiment = md_project.add_experiment(
    name="NaCl", timestep=0.002, units='metal', temperature=1200.0, simulation_data=NaCl_data
)
KCl_experiment = md_project.add_experiment(
    name="KCl", timestep=0.002, units='metal', temperature=1200.0, simulation_data=KCl_data
)

## Structural Information

The first thing we want to study is how the structure of these salts compare. Let's do so by comparing the RDFs, ADFs, coordination numbers, and potential of mean force values for the dominant interactions in each system.

For each of these computations we will directly call the project. This is simply a faster way of performing the same computation on all of the experiments at one time. What is returns is a dictionary of information for each experiment. We will demonstrate here how one can use this to extract relevant information.

In [ ]:
RDF_Data = md_project.run.RadialDistributionFunction(number_of_configurations=500, cutoff=15.0)

In [ ]:
NaCl_experiment.run.CoordinationNumbers(
    rdf_data=RDF_Data['NaCl'], savgol_window_length=111, savgol_order=9, number_of_shells=3
)

In [ ]:
KCl_experiment.run.CoordinationNumbers(
    rdf_data=RDF_Data['KCl'], savgol_window_length=111, savgol_order=7, number_of_shells=2
)

In [ ]:
KCl_experiment.run.PotentialOfMeanForce(
    rdf_data=RDF_Data['KCl'], savgol_window_length=111, savgol_order=7, number_of_shells=2
)

In [ ]:
radii = RDF_Data['NaCl']['Na_Cl']['x']
NaCl_rdf = RDF_Data['NaCl']['Na_Cl']['y']
KCl_rdf = RDF_Data['KCl']['Cl_K']['y']

In [ ]:
plt.plot(radii, NaCl_rdf, '--', label='NaCl')
plt.plot(radii, KCl_rdf, '-.', label='KCl')
plt.legend()
plt.xlabel("Radius / nm")
plt.ylabel("g(r)")
plt.show()

We can see that the peak heights of the two salts are very similar whilst the position of the NaCl is left-shifted with respect to the KCl. This suggests that the Na and Cl ions are closer together than the K and Cl ions in their respective systems. This is consistent with the fact that Potassium atoms are larger than Sodium atoms and therefore would equilibrate at a greater separation. Now let's further investigate the peak height by looking at coordination numbers.

In [ ]:
data = md_project.run.GreenKuboDiffusionCoefficients(data_range=499, correlation_time=10)

In [ ]:
data["NaCl"].data_dict["Na"]["diffusion_coefficient"]

In [ ]:
data["NaCl"].data_dict["Cl"]["diffusion_coefficient"]

In [ ]:
md_project.experiments.NaCl.species["Na"].charge = 1
md_project.experiments.NaCl.species["Cl"].charge = -1
md_project.experiments.KCl.species["K"].charge = 1
md_project.experiments.KCl.species["Cl"].charge = -1

In [ ]:
md_project.run.GreenKuboIonicConductivity(data_range=506)